<a href="https://colab.research.google.com/github/PRIYANKA-MULLU/PRIYANKA-MULLU/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1A)
Averaging validation accuracy across multiple splits can indeed give you more consistent and reliable results when evaluating the performance of a machine learning model. This practice is commonly known as cross-validation, and it helps address some of the variability that can arise when you randomly split your dataset into a training set and a validation set.

Here's why averaging validation accuracy across multiple splits can be beneficial:

***Reduced Variance:*** By performing multiple splits and averaging the results, you reduce the impact of a single random split that may have produced an unusually high or low validation accuracy. This helps you get a more stable estimate of your model's performance.

***Better Generalization Assessment:*** Cross-validation provides a more robust estimate of how well your model is likely to generalize to unseen data. It simulates the model's performance on different subsets of the data, which can give you a better understanding of its true capabilities.

***Utilizes the Entire Dataset:*** In k-fold cross-validation, for example, you partition your dataset into k subsets (folds). You train and validate your model k times, using a different fold as the validation set in each iteration. This ensures that your model gets evaluated on the entire dataset, not just a single random split.

***Identifying Overfitting:*** Cross-validation can help you detect overfitting more effectively. If your model performs well on the training data but poorly on the validation data across multiple splits, it's a sign that the model might be overfitting.

Common cross-validation techniques include k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation, among others. The choice of which method to use depends on your specific dataset and problem.

In summary, averaging validation accuracy across multiple splits through cross-validation is a valuable technique for obtaining more consistent and reliable estimates of your model's performance. It helps you make more informed decisions about your model's hyperparameters, architecture, and overall suitability for the task at hand.






2A)Cross-validation, while providing a more reliable estimate of a model's performance compared to a single train-test split, still does not directly estimate the test accuracy. Instead, it gives you a more accurate estimate of how well your model is likely to perform on unseen data, which can be a valuable proxy for the true test accuracy.

Here's why cross-validation does not directly estimate test accuracy:

***No Substitute for True Test Data: ***The ultimate measure of a model's performance is how well it performs on completely unseen data, which is represented by the test set. Cross-validation uses different subsets of your training data as validation sets, but it still relies on data that your model has seen during training. The test set is meant to be entirely separate and unseen during model development.

***Overfitting Concerns:*** While cross-validation can help you assess how well your model generalizes to different subsets of the training data, it's not immune to overfitting. If you perform hyperparameter tuning or model selection based on cross-validation results, you may inadvertently overfit your model to the validation sets used during the process.

***Data Leakage Risk:*** In some cases, information leakage from the training data to the validation sets during cross-validation can occur, especially when preprocessing steps are not correctly applied. This can lead to overly optimistic estimates of model performance.

To get an accurate estimate of test accuracy, you should reserve a completely independent and unseen dataset (the test set) that your model has never encountered during training or validation. You evaluate your trained model on this test set to assess its true performance on new, unseen examples.

Cross-validation remains a crucial tool in machine learning for model selection, hyperparameter tuning, and gaining insights into a model's generalization performance. It provides a more reliable estimate of how well your model might perform on new data than a single train-test split, but it is not a replacement for evaluating your model on a dedicated test set to obtain an accurate estimate of test accuracy.







3A)In the context of cross-validation or any technique involving randomization, the number of iterations or folds can have an impact on the estimate of a model's performance. Specifically, the effect of the number of iterations can influence the reliability and precision of the estimate. However, there's no universal rule that more iterations always result in a better estimate. The relationship between the number of iterations and the estimate depends on several factors:

***Bias-Variance Tradeoff:*** Increasing the number of iterations generally reduces the variance of the estimate. With more iterations or folds, you get a more stable and less variable estimate of performance because you are averaging the results over multiple subsamples of your data. This can be especially useful when working with limited data or when the data is noisy.

***Computational Cost:*** Each additional iteration or fold increases the computational cost. If you have a very large dataset or if model training and evaluation are computationally expensive, using a very high number of iterations may not be practical. You'll need to strike a balance between computational resources and the precision of your estimate.

***Tradeoff with Bias: ***While increasing the number of iterations reduces variance, it can potentially introduce bias. For example, in k-fold cross-validation with a small value of k (e.g., 2 or 3), you might have too few validation folds, leading to a biased estimate because your model doesn't see enough diverse data. On the other hand, using a very high value of k can lead to small training sets in each fold, potentially biasing the estimate in another way.

***Stability of the Model:*** If your model has high variance and is sensitive to the data it's trained on, increasing the number of iterations may not necessarily provide a better estimate. In such cases, you might need to address the model's instability through other means, such as regularization or feature engineering.

In practice, the choice of the number of iterations (e.g., the number of folds in k-fold cross-validation) is often guided by empirical experimentation and domain knowledge. It's common to use values like 5 or 10 for k in k-fold cross-validation, as they strike a reasonable balance between variance reduction and computational efficiency. However, if you have a specific reason to believe that more or fewer iterations are appropriate for your problem, you can adjust the number accordingly.

Ultimately, the goal is to choose a reasonable number of iterations that provides a reliable estimate of model performance without introducing excessive bias or computational cost.






4A)Increasing the number of iterations (e.g., in cross-validation) can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it may not completely solve the underlying issues associated with small datasets. Here's how increasing the number of iterations can affect the situation:

***Reducing Variability: ***With more iterations or folds, you'll be able to evaluate your model on different subsets of the data, which can help reduce the variability in your performance estimates. This can make your estimate of model performance more stable and reliable.

***Using the Data More Effectively:*** By repeatedly using the available data in different splits, you can maximize the utilization of your limited dataset. This can provide more opportunities for your model to learn and generalize better, especially when you have a small amount of data.

However, there are some important considerations:

***Bias vs. Variance Tradeoff:*** Increasing the number of iterations does reduce the variance of your estimate, but it can also introduce bias if you're using a small dataset. For instance, if you perform 10-fold cross-validation on a dataset with only 100 samples, each fold has only 10 samples for validation, which might not be representative of the overall data distribution. This can lead to a biased estimate.

***Limited Data Issue:*** Small datasets inherently have limitations. No amount of iteration can fundamentally change the fact that you have a limited amount of information to train and evaluate your model. If your dataset is too small, you might face challenges related to model overfitting, high uncertainty in estimates, and difficulties in capturing complex patterns.

***Model Selection:*** When working with very small datasets, it's essential to choose simpler models or models with regularization to avoid overfitting. No amount of cross-validation iterations can make up for a model that is too complex for the available data.

***Data Augmentation:*** In some cases, data augmentation techniques can help artificially increase the effective size of your dataset by generating new examples based on the existing ones. This can be especially useful when dealing with small datasets.

In summary, increasing the number of iterations

